In [2]:
from dataset import Covers80DatasetPerformanceChunks
from model import Model
import torch
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
%matplotlib inline

In [35]:
excluded_transforms = [
    "_PITCH_SHIFT_0", "_PITCH_SHIFT_1", "_PITCH_SHIFT_2", "_PITCH_SHIFT_3", "_PITCH_SHIFT_4",
    "_PITCH_SHIFT_5", "_PITCH_SHIFT_6", "_PITCH_SHIFT_7",
    "_TIME_STRETCH_0", "_TIME_STRETCH_1", "_TIME_STRETCH_2", "_TIME_STRETCH_3", "_TIME_STRETCH_4",
    "_TIME_STRETCH_5", "_TIME_STRETCH_6", "_TIME_STRETCH_7",  "_TIME_STRETCH_8",
]
dataset = Covers80DatasetPerformanceChunks(
    root_dir="/Users/pasinduwijesena/Documents/university/research/experiments/data/covers80_cqt", 
    excluded_transforms=excluded_transforms, 
    isolated_performance_index=0)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, num_workers=2, shuffle=False)

Using only /Users/pasinduwijesena/Documents/university/research/experiments/data/covers80_cqt/A_Whiter_Shade_Of_Pale/annie_lennox+Medusa+03-A_Whiter_Shade_Of_Pale.npy


In [12]:
def plot(data):
    data = data.transpose()
    plt.figure(figsize=(16,8))
    librosa.display.specshow(data, x_axis="time", y_axis="cqt_note")
    plt.show() 

In [13]:
def preparePlot(data, expected):
    maximum = np.argmax(data, axis=1)
    frames = np.zeros(data.shape, dtype=np.float32)
    for (step, index) in enumerate(maximum):
        frames[step,index] = 1.0
    for (step, index) in enumerate(expected):
        frames[step,index] += 0.5
    return frames

In [36]:
dataset_iter = iter(dataloader)

In [37]:
(X,Y) = next(dataset_iter)
print(X.shape)

torch.Size([100, 335, 48])


In [71]:
(X2,Y2) = next(dataset_iter)

StopIteration: 

In [ ]:
print(X.shape)

In [14]:
def getAccuracy(data, expected):
    maximum = np.argmax(data, axis=1)
    correct = 0.0
    incorrect = 0.0
    for (step, index) in enumerate(maximum):
        if expected[step] == index:
            correct += 1
        else:
            incorrect += 1
    return (correct/ (correct+incorrect))

In [15]:
def evaluate(model, dataset):
    accuracy = 0.0
    divisor = 0
    model = model.eval()
    with torch.no_grad():
        for (sequence, next_frame) in (dataset):
            next_pred_frame = model(sequence)
            accuracy += getAccuracy(next_pred_frame, next_frame)
            divisor += 1
    print("Accuracy: {}%".format(100*accuracy/divisor))

In [ ]:
snapshots = [2,4,8,16,32,64,128,192]
# snapshots = [2]

In [ ]:
# with the original training dataset
for snapshot in snapshots:
    snapshot_path = "/Users/pasinduwijesena/Documents/university/research/experiments/comparing-rnn-params/model/snapshots/exp12/snapshot-{}.pytorch".format(snapshot)
    data =torch.load(snapshot_path)
    model = Model(input_size=48, hidden_size=64)
    model.load_state_dict(data['model_state_dict'])
    
    print("Evaluating with snapshot {}".format(snapshot))
    evaluate(model, dataloader)

In [ ]:
# with a few minor augmentations
for snapshot in snapshots:
    snapshot_path = "/Users/pasinduwijesena/Documents/university/research/experiments/comparing-rnn-params/model/snapshots/exp12/snapshot-{}.pytorch".format(snapshot)
    data =torch.load(snapshot_path)
    model = Model(input_size=48, hidden_size=64)
    model.load_state_dict(data['model_state_dict'])
    
    print("Evaluating with snapshot {}".format(snapshot))
    evaluate(model, dataloader2)

In [ ]:
# with all augmentations
for snapshot in snapshots:
    snapshot_path = "/Users/pasinduwijesena/Documents/university/research/experiments/comparing-rnn-params/model/snapshots/exp12/snapshot-{}.pytorch".format(snapshot)
    data =torch.load(snapshot_path)
    model = Model(input_size=48, hidden_size=64)
    model.load_state_dict(data['model_state_dict'])
    
    print("Evaluating with snapshot {}".format(snapshot))
    evaluate(model, dataloader3)